In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
import numpy as np

In [1]:
def gamma_variate(t, tmax, b, delay):
    """
    Gamma variate function for modeling the hemodynamic response.
    """
    t_adjusted = np.maximum(t - delay, 0)
    gv = (t_adjusted / tmax)**(b * tmax) * np.exp(b * (tmax - t_adjusted))
    return gv


In [2]:
import nibabel as nib

def load_nifti(file_path):
    # Load a NIfTI file
    nifti_data = nib.load(file_path)
    return nifti_data.get_fdata()


In [3]:
def flatten_spatial_dimensions(nifti_data):
    # nifti_data is a 4D numpy array: (x, y, z, time)
    n_timepoints = nifti_data.shape[-1]
    flattened_data = []

    for t in range(n_timepoints):
        timepoint_data = nifti_data[:,:,:,t]
        flattened_data.append(timepoint_data.flatten())

    return np.array(flattened_data)


In [ ]:
patient_datasets =   # List of file paths to patient datasets

preprocessed_data = []
for file_path in patient_datasets:
    nifti_data = load_nifti(file_path)
    flattened_data = flatten_spatial_dimensions(nifti_data)
    # Additional preprocessing (e.g., normalization) can be applied here
    preprocessed_data.append(flattened_data)


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
import numpy as np

# Assuming preprocessed_data is a list of 2D arrays (time x voxels) for each patient

best_params_per_patient = []
k = 5  # Number of folds in k-fold cross-validation
l1_penalty = 0.01  # L1 regularization strength

for patient_data in preprocessed_data:  # Loop over each patient's data
    param_grid = {
        'tmax': np.linspace(start_tmax, end_tmax, num=...),
        'b': np.linspace(start_b, end_b, num=...)
    }
    best_params = {'tmax': None, 'b': None}
    best_score = float('-inf')

    kfold = KFold(n_splits=k, shuffle=True)

    for tmax in param_grid['tmax']:
        for b in param_grid['b']:
            scores = []
            for train_index, test_index in kfold.split(patient_data):
                # Generate predicted time series for current parameters
                predicted_time_series = gamma_variate(t, tmax, b, delay)

                glm_model = Lasso(alpha=l1_penalty)  # L1 regularization
                glm_model.fit(patient_data[train_index], predicted_time_series[train_index])
                score = glm_model.score(patient_data[test_index], predicted_time_series[test_index])

                scores.append(score)

            avg_score = np.mean(scores)
            if avg_score > best_score:
                best_score = avg_score
                best_params['tmax'] = tmax
                best_params['b'] = b

    best_params_per_patient.append(best_params)

# best_params_per_patient now holds the best parameters for each patient
